In [ ]:
# 快速开始

[NautilusTrader](https://nautilustrader.io/docs/) 高性能算法交易平台和事件驱动回测器教程。

[在 GitHub 上查看源码](https://github.com/nautechsystems/nautilus_trader/blob/develop/docs/getting_started/quickstart.ipynb)。


In [ ]:
## 概述

本快速开始教程将逐步介绍如何使用 FX 数据运行 NautilusTrader 回测。
为了支持这一点，我们提供了一些使用标准 Nautilus 持久化格式（Parquet）的预加载测试数据。


In [ ]:
## 先决条件
- 已安装 Python 3.11+
- 已安装 [NautilusTrader](https://pypi.org/project/nautilus_trader/) 最新版本 (`pip install -U nautilus_trader`)
- 已安装 [JupyterLab](https://jupyter.org/) 或类似软件 (`pip install -U jupyterlab`)


In [ ]:
## 1. 获取示例数据

为了节省时间，我们准备了一个脚本来将示例数据加载到 Nautilus 格式中，以便与此示例一起使用。
首先，通过运行下一个单元格来下载和加载数据（这应该需要大约 1-2 分钟）：

```bash
!apt-get update && apt-get install curl -y
!curl https://raw.githubusercontent.com/nautechsystems/nautilus_data/main/nautilus_data/hist_data_to_catalog.py | python -
```

有关如何将数据加载到 Nautilus 中的更多详细信息，请参阅[加载外部数据](https://nautilustrader.io/docs/latest/concepts/data#loading-data)指南。


In [ ]:
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Quantity
from nautilus_trader.model import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog


In [ ]:
## 2. 设置 Parquet 数据目录

如果一切正常，您应该能够在目录中看到一个 EUR/USD 工具。


In [ ]:
# 您也可以使用相对路径，例如 `ParquetDataCatalog("./catalog")`，
# 例如，如果您在文档中的数据设置后运行此笔记本。
# catalog = ParquetDataCatalog("./catalog")
catalog = ParquetDataCatalog.from_env()
catalog.instruments()


In [ ]:
## 3. 编写交易策略

NautilusTrader 包含许多内置指标，在这个例子中我们将使用 MACD 指标来构建一个简单的交易策略。

您可以在[这里了解更多关于 MACD 的信息](https://www.investopedia.com/terms/m/macd.asp)，这个指标仅作为示例，不期望有任何阿尔法。还有一种注册指标以接收特定数据类型的方法，但在这个例子中，我们在 `on_quote_tick` 方法中手动将接收到的 `QuoteTick` 传递给指标。


In [ ]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model import InstrumentId
from nautilus_trader.model import Position
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.events import PositionOpened
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.trading.strategy import StrategyConfig


class MACDConfig(StrategyConfig):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 1_000_000
    entry_threshold: float = 0.00010


class MACDStrategy(Strategy):
    def __init__(self, config: MACDConfig):
        super().__init__(config=config)
        # 我们的"交易信号"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period, slow_period=config.slow_period, price_type=PriceType.MID
        )

        self.trade_size = Quantity.from_int(config.trade_size)

        # 便利性
        self.position: Position | None = None

    def on_start(self):
        self.subscribe_quote_ticks(instrument_id=self.config.instrument_id)

    def on_stop(self):
        self.close_all_positions(self.config.instrument_id)
        self.unsubscribe_quote_ticks(instrument_id=self.config.instrument_id)

    def on_quote_tick(self, tick: QuoteTick):
        # 您可以注册指标以自动接收报价 tick 更新，
        # 这里我们手动更新指标以展示可用的灵活性。
        self.macd.handle_quote_tick(tick)

        if not self.macd.initialized:
            return  # 等待指标预热

        # self._log.info(f"{self.macd.value=}:%5d")
        self.check_for_entry()
        self.check_for_exit()

    def on_event(self, event: Event):
        if isinstance(event, PositionOpened):
            self.position = self.cache.position(event.position_id)

    def check_for_entry(self):
        # 如果 MACD 线高于我们的入场阈值，我们应该做多
        if self.macd.value > self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.LONG:
                return  # 已经做多

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=self.trade_size,
            )
            self.submit_order(order)
        # 如果 MACD 线低于我们的入场阈值，我们应该做空
        elif self.macd.value < -self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.SHORT:
                return  # 已经做空

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=self.trade_size,
            )
            self.submit_order(order)

    def check_for_exit(self):
        # 如果 MACD 线高于零，则在做空时退出
        if self.macd.value >= 0.0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)
        # 如果 MACD 线低于零，则在做多时退出
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)

    def on_dispose(self):
        pass  # 不做其他事情


In [ ]:
## 配置回测

现在我们有了交易策略和数据，可以开始配置回测运行。Nautilus 使用 `BacktestNode` 来编排回测运行，这需要一些设置。这一开始可能看起来有点复杂，但这是 Nautilus 追求的功能所必需的。

要配置 `BacktestNode`，我们首先需要创建一个 `BacktestRunConfig` 实例，配置回测的以下（最小）方面：

- `engine`：代表我们核心系统的回测引擎，它也将包含我们的策略
- `venues`：回测中可用的模拟场所（交易所或经纪商）
- `data`：我们想要执行回测的输入数据

还有许多其他可配置的功能，这些将在文档的后面部分描述，现在这将让我们运行起来。

## 4. 配置场所

首先，我们创建一个场所配置。对于这个例子，我们将创建一个模拟的 FX ECN。
场所需要一个作为 ID 的名称（在这种情况下是 `SIM`），以及一些基本配置，例如账户类型（`CASH` vs `MARGIN`）、可选的基础货币和起始余额。

:::note
FX 交易通常在保证金上使用不可交割远期、掉期或 CFD 类型的工具进行。
:::


In [ ]:
venue = BacktestVenueConfig(
    name="SIM",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USD",
    starting_balances=["1_000_000 USD"]
)


In [ ]:
## 5. 配置数据

我们需要了解我们想要加载数据的工具，我们可以使用 `ParquetDataCatalog` 来实现这一点。


In [ ]:
instruments = catalog.instruments()
instruments


In [ ]:
接下来，我们需要为回测配置数据。Nautilus 在为回测加载数据方面建立得非常灵活，但这也意味着需要一些配置。

对于每种 tick 类型（和工具），我们添加一个 `BacktestDataConfig`。在这个例子中，我们只是为我们的 EUR/USD 工具添加 `QuoteTick`：


In [ ]:
from nautilus_trader.model import QuoteTick


data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=QuoteTick,
    instrument_id=instruments[0].id,
    end_time="2020-01-10",
)


In [ ]:
## 6. 配置引擎

然后，我们需要一个 `BacktestEngineConfig`，它代表我们核心交易系统的配置。
这里我们需要传递我们的交易策略，我们也可以调整日志级别并配置许多其他组件（但是，使用默认值也很好）：

通过 `ImportableStrategyConfig` 添加策略，它能够从任意文件或用户包中导入策略。在这个例子中，我们的 `MACDStrategy` 在当前模块中定义，Python 将其称为 `__main__`。


In [ ]:
# NautilusTrader 目前超过了 Jupyter notebook 日志记录（标准输出）的速率限制，
# 这就是为什么将 `log_level` 设置为 "ERROR"。如果您降低此级别以查看
# 更多日志记录，那么笔记本将在单元格执行期间挂起。目前正在
# 调查一个修复方案，该方案涉及提高 Jupyter 的配置速率限制，
# 或者限制来自 Nautilus 的日志刷新。
# https://github.com/jupyterlab/jupyterlab/issues/12845
# https://github.com/deshaw/jupyterlab-limit-output
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDStrategy",
            config_path="__main__:MACDConfig",
            config={
              "instrument_id": instruments[0].id,
              "fast_period": 12,
              "slow_period": 26,
            },
        )
    ],
    logging=LoggingConfig(log_level="ERROR"),
)


In [ ]:
## 7. 运行回测

我们现在可以将各种配置片段传递给 `BacktestRunConfig`。这个对象现在包含我们回测的完整配置。


In [ ]:
config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
)


In [ ]:
`BacktestNode` 类将编排回测运行。配置和执行之间分离的原因是 `BacktestNode`，它能够运行多个配置（不同的参数或数据批次）。我们现在准备运行一些回测。


In [ ]:
from nautilus_trader.backtest.results import BacktestResult


node = BacktestNode(configs=[config])

 # 同步运行一个或多个配置
results: list[BacktestResult] = node.run()


In [ ]:
## 8. 分析结果


In [ ]:
现在运行完成，我们还可以通过使用运行配置 ID 直接查询 `BacktestNode` 内部使用的 `BacktestEngine`。

引擎可以提供额外的报告和信息。


In [ ]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model import Venue


engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()


In [ ]:
engine.trader.generate_positions_report()


In [ ]:
engine.trader.generate_account_report(Venue("SIM"))
